In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
#to draw grid lines on Lambert Conformal projection; 
#CREDIT ajdawson on GitHub https://gist.github.com/ajdawson/dd536f786741e987ae4e
from copy import copy
import shapely.geometry as sgeom
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])


def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])
    

def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])
def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible:    
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [ ]:
ds1=xr.open_dataset(dir+'station_numbers_95.nc')
lats_station=ds1['lats'].values.tolist()#use these lists for plotting
lons_station=ds1['lons'].values.tolist()
stations_station=ds1['stations'].values.tolist()
thresholds_fall_99=ds1['thresholds_fall_99']/10.
thresholds_fall_99=thresholds_fall_99.values.tolist()
thresholds_winter_99=ds1['thresholds_winter_99']/10.
thresholds_winter_99=thresholds_winter_99.values.tolist()
thresholds_spring_99=ds1['thresholds_spring_99']/10.
thresholds_spring_99=thresholds_spring_99.values.tolist()
thresholds_summer_99=ds1['thresholds_summer_99']/10.
thresholds_summer_99=thresholds_summer_99.values.tolist()

thresholds_fall_95=ds1['thresholds_fall_95']/10.
thresholds_fall_95=thresholds_fall_95.values.tolist()
thresholds_winter_95=ds1['thresholds_winter_95']/10.
thresholds_winter_95=thresholds_winter_95.values.tolist()
thresholds_spring_95=ds1['thresholds_spring_95']/10.
thresholds_spring_95=thresholds_spring_95.values.tolist()
thresholds_summer_95=ds1['thresholds_summer_95']/10.
thresholds_summer_95=thresholds_summer_95.values.tolist()

In [ ]:
yrs_neusa=np.arange(1979,2020,1)
annual_rate_t1_list=[]
annual_rate_t2_list=[]
annual_rate_t3_list=[]
annual_rate_t4_list=[]
annual_rate_t5_list=[]
annual_rate_t6_list=[]
annual_rate_t7_list=[]
annual_rate_t8_list=[]
annual_rate_t9_list=[]
annual_rate_t10_list=[]
annual_rate_t11_list=[]

annual_rate_ar_list=[]
annual_rate_tc_list=[]
annual_rate_other_list=[]

annual_intensity_t1_list=[]
annual_intensity_t2_list=[]
annual_intensity_t3_list=[]
annual_intensity_t4_list=[]
annual_intensity_t5_list=[]
annual_intensity_t6_list=[]
annual_intensity_t7_list=[]
annual_intensity_t8_list=[]
annual_intensity_t9_list=[]
annual_intensity_t10_list=[]
annual_intensity_t11_list=[]

annual_intensity_ar_list=[]
annual_intensity_tc_list=[]
annual_intensity_other_list=[]

years_t1_list=[]
years_t2_list=[]
years_t3_list=[]
years_t4_list=[]
years_t5_list=[]
years_t6_list=[]
years_t7_list=[]
years_t8_list=[]
years_t9_list=[]
years_t10_list=[]
years_t11_list=[]

years_ar_list=[]
years_tc_list=[]
years_other_list=[]


lats_t1_list=[]
lats_t2_list=[]
lats_t3_list=[]
lats_t4_list=[]
lats_t5_list=[]
lats_t6_list=[]
lats_t7_list=[]
lats_t8_list=[]
lats_t9_list=[]
lats_t10_list=[]
lats_t11_list=[]

lats_ar_list=[]
lats_tc_list=[]
lats_other_list=[]
lats_linked_list=[]

lons_t1_list=[]
lons_t2_list=[]
lons_t3_list=[]
lons_t4_list=[]
lons_t5_list=[]
lons_t6_list=[]
lons_t7_list=[]
lons_t8_list=[]
lons_t9_list=[]
lons_t10_list=[]
lons_t11_list=[]

lons_ar_list=[]
lons_tc_list=[]
lons_other_list=[]
lons_linked_list=[]

stations_t1_list=[]
stations_t2_list=[]
stations_t3_list=[]
stations_t4_list=[]
stations_t5_list=[]
stations_t6_list=[]
stations_t7_list=[]
stations_t8_list=[]
stations_t9_list=[]
stations_t10_list=[]
stations_t11_list=[]

stations_tc_list=[]
stations_ar_list=[]
stations_other_list=[]
season='winter'
for i in range(1,len(yrs_neusa)):
    print(i)
    if i>1:
        ds.close()
    year=yrs_neusa[i]
    #ds=xr.open_dataset(dir+'climo_neusa_precip_days_stats_'+str(year)+'.nc')
    ds=xr.open_dataset(dir+'climo_neusa_ar_'+season+'_'+str(year)+'.nc')
    #print(ds)

    lats_pure_ar=ds['lats_pure_ar'].values.tolist()
    lons_pure_ar=ds['lons_pure_ar'].values.tolist()
    lons_pure_ar=[x-360. for x in lons_pure_ar]
    obs_pure_ar=ds['obs_pure_ar'].values.tolist()
    dates_pure_ar=ds['dates_pure_ar'].values
    stations_pure_ar=ds['stations_pure_ar'].values.tolist()#filenames

    lats_pure_tc=ds['lats_pure_tc'].values.tolist()
    lons_pure_tc=ds['lons_pure_tc'].values.tolist()
    lons_pure_tc=[x-360. for x in lons_pure_tc]
    obs_pure_tc=ds['obs_pure_tc'].values.tolist()
    dates_pure_tc=ds['dates_pure_tc'].values
    stations_pure_tc=ds['stations_pure_tc'].values.tolist()

    lats_pure_extreme_ivt=ds['lats_pure_extreme_ivt'].values.tolist()
    lons_pure_extreme_ivt=ds['lons_pure_extreme_ivt'].values.tolist()
    lons_pure_extreme_ivt=[x-360. for x in lons_pure_extreme_ivt]
    obs_pure_extreme_ivt=ds['obs_pure_extreme_ivt'].values.tolist()
    dates_pure_extreme_ivt=ds['dates_pure_extreme_ivt'].values
    stations_pure_extreme_ivt=ds['stations_pure_extreme_ivt'].values.tolist()

    lats_tc_linked_ar=ds['lats_tc_linked_ar'].values.tolist()
    lons_tc_linked_ar=ds['lons_tc_linked_ar'].values.tolist()
    lons_tc_linked_ar=[x-360. for x in lons_tc_linked_ar]
    obs_tc_linked_ar=ds['obs_tc_linked_ar'].values.tolist()
    dates_tc_linked_ar=ds['dates_tc_linked_ar'].values
    stations_tc_linked_ar=ds['stations_tc_linked_ar'].values.tolist()

    lats_tc_remnant_linked_ar=ds['lats_tc_remnant_linked_ar'].values.tolist()
    lons_tc_remnant_linked_ar=ds['lons_tc_remnant_linked_ar'].values.tolist()
    lons_tc_remnant_linked_ar=[x-360. for x in lons_tc_remnant_linked_ar]
    obs_tc_remnant_linked_ar=ds['obs_tc_remnant_linked_ar'].values.tolist()
    dates_tc_remnant_linked_ar=ds['dates_tc_remnant_linked_ar'].values
    stations_tc_remnant_linked_ar=ds['stations_tc_remnant_linked_ar'].values.tolist()

    lats_tc_linked_ivt=ds['lats_tc_linked_ivt'].values.tolist()
    lons_tc_linked_ivt=ds['lons_tc_linked_ivt'].values.tolist()
    lons_tc_linked_ivt=[x-360. for x in lons_tc_linked_ivt]
    obs_tc_linked_ivt=ds['obs_tc_linked_ivt'].values.tolist()
    dates_tc_linked_ivt=ds['dates_tc_linked_ivt'].values
    stations_tc_linked_ivt=ds['stations_tc_linked_ivt'].values.tolist()

    lats_tc_remnant_linked_ivt=ds['lats_tc_remnant_linked_ivt'].values.tolist()
    lons_tc_remnant_linked_ivt=ds['lons_tc_remnant_linked_ivt'].values.tolist()
    lons_tc_remnant_linked_ivt=[x-360. for x in lons_tc_remnant_linked_ivt]
    obs_tc_remnant_linked_ivt=ds['obs_tc_remnant_linked_ivt'].values.tolist()
    dates_tc_remnant_linked_ivt=ds['dates_tc_remnant_linked_ivt'].values
    stations_tc_remnant_linked_ivt=ds['stations_tc_remnant_linked_ivt'].values.tolist()

    lats_tc_remnants=ds['lats_tc_remnants'].values.tolist()
    lons_tc_remnants=ds['lons_tc_remnants'].values.tolist()
    lons_tc_remnants=[x-360. for x in lons_tc_remnants]
    obs_tc_remnants=ds['obs_tc_remnants'].values.tolist()
    dates_tc_remnants=ds['dates_tc_remnants'].values
    stations_tc_remnants=ds['stations_tc_remnants'].values.tolist()

    lats_tc_ar_combo=ds['lats_tc_ar_combo'].values.tolist()
    lons_tc_ar_combo=ds['lons_tc_ar_combo'].values.tolist()
    lons_tc_ar_combo=[x-360. for x in lons_tc_ar_combo]
    obs_tc_ar_combo=ds['obs_tc_ar_combo'].values.tolist()
    dates_tc_ar_combo=ds['dates_tc_ar_combo'].values
    stations_tc_ar_combo=ds['stations_tc_ar_combo'].values.tolist()

    lats_tc_remnant_ar_combo=ds['lats_tc_remnant_ar_combo'].values.tolist()
    lons_tc_remnant_ar_combo=ds['lons_tc_remnant_ar_combo'].values.tolist()
    lons_tc_remnant_ar_combo=[x-360. for x in lons_tc_remnant_ar_combo]
    obs_tc_remnant_ar_combo=ds['obs_tc_remnant_ar_combo'].values.tolist()
    dates_tc_remnant_ar_combo=ds['dates_tc_remnant_ar_combo'].values
    stations_tc_remnant_ar_combo=ds['stations_tc_remnant_ar_combo'].values.tolist()

    lats_other=ds['lats_other'].values.tolist()
    lons_other=ds['lons_other'].values.tolist()
    lons_other=[x-360. for x in lons_other]
    obs_other=ds['obs_other'].values.tolist()
    dates_other=ds['dates_other'].values
    stations_other=ds['stations_other'].values.tolist()
    
    zipped_t1=list(zip(dates_pure_ar,lats_pure_ar,lons_pure_ar,obs_pure_ar,stations_pure_ar))
    zipped_t2=list(zip(dates_tc_linked_ar,lats_tc_linked_ar,lons_tc_linked_ar,obs_tc_linked_ar,stations_tc_linked_ar))
    zipped_t3=list(zip(dates_tc_remnant_linked_ar,lats_tc_remnant_linked_ar,lons_tc_remnant_linked_ar,obs_tc_remnant_linked_ar,stations_tc_remnant_linked_ar))
    zipped_t4=list(zip(dates_pure_tc,lats_pure_tc,lons_pure_tc,obs_pure_tc,stations_pure_tc))
    zipped_t5=list(zip(dates_tc_ar_combo,lats_tc_ar_combo,lons_tc_ar_combo,obs_tc_ar_combo,stations_tc_ar_combo))
    zipped_t6=list(zip(dates_tc_remnant_ar_combo,lats_tc_remnant_ar_combo,lons_tc_remnant_ar_combo,obs_tc_remnant_ar_combo,stations_tc_remnant_ar_combo))
    zipped_t7=list(zip(dates_tc_remnants,lats_tc_remnants,lons_tc_remnants,obs_tc_remnants,stations_tc_remnants))
    zipped_t8=list(zip(dates_pure_extreme_ivt,lats_pure_extreme_ivt,lons_pure_extreme_ivt,obs_pure_extreme_ivt,stations_pure_extreme_ivt))
    zipped_t9=list(zip(dates_tc_linked_ivt,lats_tc_linked_ivt,lons_tc_linked_ivt,obs_tc_linked_ivt,stations_tc_linked_ivt))
    zipped_t10=list(zip(dates_tc_remnant_linked_ivt,lats_tc_remnant_linked_ivt,lons_tc_remnant_linked_ivt,obs_tc_remnant_linked_ivt,stations_tc_remnant_linked_ivt))
    zipped_t11=list(zip(dates_other,lats_other,lons_other,obs_other,stations_other))

    for j in range(len(stations_station)):
        station=stations_station[j]
        thresh=thresholds_winter_95[j]
        lat=lats_station[j]
        lon=lons_station[j]
        select_station_t1=[x for x in zipped_t1 if x[4]==station and x[3]>=thresh]
        select_station_t2=[x for x in zipped_t2 if x[4]==station and x[3]>=thresh]
        select_station_t3=[x for x in zipped_t3 if x[4]==station and x[3]>=thresh]
        select_station_t4=[x for x in zipped_t4 if x[4]==station and x[3]>=thresh]
        select_station_t5=[x for x in zipped_t5 if x[4]==station and x[3]>=thresh]
        select_station_t6=[x for x in zipped_t6 if x[4]==station and x[3]>=thresh]
        select_station_t7=[x for x in zipped_t7 if x[4]==station and x[3]>=thresh]
        select_station_t8=[x for x in zipped_t8 if x[4]==station and x[3]>=thresh]
        select_station_t9=[x for x in zipped_t9 if x[4]==station and x[3]>=thresh]
        select_station_t10=[x for x in zipped_t10 if x[4]==station and x[3]>=thresh]
        select_station_t11=[x for x in zipped_t11 if x[4]==station and x[3]>=thresh]

        select_station_ar=select_station_t1+select_station_t2+select_station_t3
        select_station_tc=select_station_t4+select_station_t5+select_station_t6+select_station_t7
        select_station_other=select_station_t8+select_station_t9+select_station_t10+select_station_t11

        if len(select_station_t1)>0:
            obs_station_t1=[x[3] for x in select_station_t1]
            total_precip_station_t1=sum(obs_station_t1)
            annual_rate_t1=total_precip_station_t1
            annual_rate_t1_list.append(annual_rate_t1)
            lats_t1_list.append(lat)
            lons_t1_list.append(lon)
            stations_t1_list.append(station)
            
            years_t1_list.append(year)

            mean_intensity_t1=float(sum(obs_station_t1))/float(len(obs_station_t1))
            annual_intensity_t1_list.append(mean_intensity_t1)
        else:
            annual_rate_t1_list.append(0)
            years_t1_list.append(year)
            stations_t1_list.append(station)

        if len(select_station_t2)>0:
            obs_station_t2=[x[3] for x in select_station_t2]
            total_precip_station_t2=sum(obs_station_t2)
            annual_rate_t2=total_precip_station_t2
            annual_rate_t2_list.append(annual_rate_t2)
            lats_t2_list.append(lat)
            lons_t2_list.append(lon)
            stations_t2_list.append(station)
            
            years_t2_list.append(year)

            mean_intensity_t2=float(sum(obs_station_t2))/float(len(obs_station_t2))
            annual_intensity_t2_list.append(mean_intensity_t2)
            
        else:
            annual_rate_t2_list.append(0)
            years_t2_list.append(year)
            stations_t2_list.append(station)

        if len(select_station_t3)>0:
            obs_station_t3=[x[3] for x in select_station_t3]
            total_precip_station_t3=sum(obs_station_t3)
            annual_rate_t3=total_precip_station_t3
            annual_rate_t3_list.append(annual_rate_t3)
            lats_t3_list.append(lat)
            lons_t3_list.append(lon)
            stations_t3_list.append(station)
            
            years_t3_list.append(year)

            mean_intensity_t3=float(sum(obs_station_t3))/float(len(obs_station_t3))
            annual_intensity_t3_list.append(mean_intensity_t3)
            
        else:
            annual_rate_t3_list.append(0)
            years_t3_list.append(year)
            stations_t3_list.append(station)

        if len(select_station_t4)>0:
            obs_station_t4=[x[3] for x in select_station_t4]
            total_precip_station_t4=sum(obs_station_t4)
            annual_rate_t4=total_precip_station_t4
            annual_rate_t4_list.append(annual_rate_t4)
            lats_t4_list.append(lat)
            lons_t4_list.append(lon)
            stations_t4_list.append(station)
            
            years_t4_list.append(year)

            mean_intensity_t4=float(sum(obs_station_t4))/float(len(obs_station_t4))
            annual_intensity_t4_list.append(mean_intensity_t4)
            
        else:
            annual_rate_t4_list.append(0)
            years_t4_list.append(year)
            stations_t4_list.append(station)

        if len(select_station_t5)>0:
            obs_station_t5=[x[3] for x in select_station_t5]
            total_precip_station_t5=sum(obs_station_t5)
            annual_rate_t5=total_precip_station_t5
            annual_rate_t5_list.append(annual_rate_t5)
            lats_t5_list.append(lat)
            lons_t5_list.append(lon)
            stations_t5_list.append(station)
            
            years_t5_list.append(year)

            mean_intensity_t5=float(sum(obs_station_t5))/float(len(obs_station_t5))
            annual_intensity_t5_list.append(mean_intensity_t5)
            
        else:
            annual_rate_t5_list.append(0)
            years_t5_list.append(year)
            stations_t5_list.append(station)

        if len(select_station_t6)>0:
            obs_station_t6=[x[3] for x in select_station_t6]
            total_precip_station_t6=sum(obs_station_t6)
            annual_rate_t6=total_precip_station_t6
            annual_rate_t6_list.append(annual_rate_t6)
            lats_t6_list.append(lat)
            lons_t6_list.append(lon)
            stations_t6_list.append(station)
            
            years_t6_list.append(year)

            mean_intensity_t6=float(sum(obs_station_t6))/float(len(obs_station_t6))
            annual_intensity_t6_list.append(mean_intensity_t6)
            
        else:
            annual_rate_t6_list.append(0)
            years_t6_list.append(year)
            stations_t6_list.append(station)

        if len(select_station_t7)>0:
            obs_station_t7=[x[3] for x in select_station_t7]
            total_precip_station_t7=sum(obs_station_t7)
            annual_rate_t7=total_precip_station_t7
            annual_rate_t7_list.append(annual_rate_t7)
            lats_t7_list.append(lat)
            lons_t7_list.append(lon)
            stations_t7_list.append(station)
            
            years_t7_list.append(year)

            mean_intensity_t7=float(sum(obs_station_t7))/float(len(obs_station_t7))
            annual_intensity_t7_list.append(mean_intensity_t7)
            
        else:
            annual_rate_t7_list.append(0)
            years_t7_list.append(year)
            stations_t7_list.append(station)

        if len(select_station_t8)>0:
            obs_station_t8=[x[3] for x in select_station_t8]
            total_precip_station_t8=sum(obs_station_t8)
            annual_rate_t8=total_precip_station_t8
            annual_rate_t8_list.append(annual_rate_t8)
            lats_t8_list.append(lat)
            lons_t8_list.append(lon)
            stations_t8_list.append(station)
            
            years_t8_list.append(year)

            mean_intensity_t8=float(sum(obs_station_t8))/float(len(obs_station_t8))
            annual_intensity_t8_list.append(mean_intensity_t8)
            
        else:
            annual_rate_t8_list.append(0)
            years_t8_list.append(year)
            stations_t8_list.append(station)

        if len(select_station_t9)>0:
            obs_station_t9=[x[3] for x in select_station_t9]
            total_precip_station_t9=sum(obs_station_t9)
            annual_rate_t9=total_precip_station_t9
            annual_rate_t9_list.append(annual_rate_t9)
            lats_t9_list.append(lat)
            lons_t9_list.append(lon)
            stations_t9_list.append(station)
            
            years_t9_list.append(year)

            mean_intensity_t9=float(sum(obs_station_t9))/float(len(obs_station_t9))
            annual_intensity_t9_list.append(mean_intensity_t9)
            
        else:
            annual_rate_t9_list.append(0)
            years_t9_list.append(year)
            stations_t9_list.append(station)

        if len(select_station_t10)>0:
            obs_station_t10=[x[3] for x in select_station_t10]
            total_precip_station_t10=sum(obs_station_t10)
            annual_rate_t10=total_precip_station_t10
            annual_rate_t10_list.append(annual_rate_t10)
            lats_t10_list.append(lat)
            lons_t10_list.append(lon)
            stations_t10_list.append(station)
            
            years_t10_list.append(year)

            mean_intensity_t10=float(sum(obs_station_t10))/float(len(obs_station_t10))
            annual_intensity_t10_list.append(mean_intensity_t10)
            
        else:
            annual_rate_t10_list.append(0)
            years_t10_list.append(year)
            stations_t10_list.append(station)

        if len(select_station_t11)>0:
            obs_station_t11=[x[3] for x in select_station_t11]
            total_precip_station_t11=sum(obs_station_t11)
            annual_rate_t11=total_precip_station_t11
            annual_rate_t11_list.append(annual_rate_t11)
            lats_t11_list.append(lat)
            lons_t11_list.append(lon)
            stations_t11_list.append(station)
            
            years_t11_list.append(year)

            mean_intensity_t11=float(sum(obs_station_t11))/float(len(obs_station_t11))
            annual_intensity_t11_list.append(mean_intensity_t11)
            
        else:
            annual_rate_t11_list.append(0)
            years_t11_list.append(year)
            stations_t11_list.append(station)

        if len(select_station_ar)>0:
            obs_station_ar=[x[3] for x in select_station_ar]
            total_precip_station_ar=sum(obs_station_ar)
            annual_rate_ar=total_precip_station_ar
            annual_rate_ar_list.append(annual_rate_ar)
            lats_ar_list.append(lat)
            lons_ar_list.append(lon)
            stations_ar_list.append(station)
            
            years_ar_list.append(year)

            mean_intensity_ar=float(sum(obs_station_ar))/float(len(obs_station_ar))
            annual_intensity_ar_list.append(mean_intensity_ar)
            
        else:
            annual_rate_ar_list.append(0)
            years_ar_list.append(year)
            stations_ar_list.append(station)

        if len(select_station_tc)>0:
            obs_station_tc=[x[3] for x in select_station_tc]
            total_precip_station_tc=sum(obs_station_tc)
            annual_rate_tc=total_precip_station_tc
            annual_rate_tc_list.append(annual_rate_tc)
            lats_tc_list.append(lat)
            lons_tc_list.append(lon)
            stations_tc_list.append(station)
            
            years_tc_list.append(year)

            mean_intensity_tc=float(sum(obs_station_tc))/float(len(obs_station_tc))
            annual_intensity_tc_list.append(mean_intensity_tc)
            
        else:
            annual_rate_tc_list.append(0)
            years_tc_list.append(year)
            stations_tc_list.append(station)

        if len(select_station_other)>0:
            obs_station_other=[x[3] for x in select_station_other]
            total_precip_station_other=sum(obs_station_other)
            annual_rate_other=total_precip_station_other
            annual_rate_other_list.append(annual_rate_other)
            lats_other_list.append(lat)
            lons_other_list.append(lon)
            stations_other_list.append(station)
            
            years_other_list.append(year)

            mean_intensity_other=float(sum(obs_station_other))/float(len(obs_station_other))
            annual_intensity_other_list.append(mean_intensity_other)
            
        else:
            annual_rate_other_list.append(0)
            years_other_list.append(year)
            stations_other_list.append(station)

In [ ]:
#@author: Michael Schramm on GitHub
#This function is derived from code originally posted by Sat Kumar Tomer
#(satkumartomer@gmail.com)
#See also: http://vsp.pnnl.gov/help/Vsample/Design_Trend_Mann_Kendall.htm
from scipy.stats import norm
import scipy.stats as st
def mk_test(x, alpha=0.05):
    n = len(x)

    # calculate S
    s = 0
    for k in range(n-1):
        for j in range(k+1, n):
            s += np.sign(x[j] - x[k])

    # calculate the unique data
    unique_x, tp = np.unique(x, return_counts=True)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else:  # there are some ties in data
        var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s > 0:
        z = (s - 1)/np.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/np.sqrt(var_s)
    else: # s == 0:
        z = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1-alpha/2)

    if (z < 0) and h:
        trend = 'decreasing'
    elif (z > 0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z

In [ ]:
zipped_t1_total_list=list(zip(years_t1_list,annual_rate_t1_list,stations_t1_list))
zipped_t2_total_list=list(zip(years_t2_list,annual_rate_t2_list,stations_t2_list))
zipped_t3_total_list=list(zip(years_t3_list,annual_rate_t3_list,stations_t3_list))
zipped_t4_total_list=list(zip(years_t4_list,annual_rate_t4_list,stations_t4_list))
zipped_t5_total_list=list(zip(years_t5_list,annual_rate_t5_list,stations_t5_list))
zipped_t6_total_list=list(zip(years_t6_list,annual_rate_t6_list,stations_t6_list))
zipped_t7_total_list=list(zip(years_t7_list,annual_rate_t7_list,stations_t7_list))
zipped_t8_total_list=list(zip(years_t8_list,annual_rate_t8_list,stations_t8_list))
zipped_t9_total_list=list(zip(years_t9_list,annual_rate_t9_list,stations_t9_list))
zipped_t10_total_list=list(zip(years_t10_list,annual_rate_t10_list,stations_t10_list))
zipped_t11_total_list=list(zip(years_t11_list,annual_rate_t11_list,stations_t11_list))

zipped_tc_total_list=list(zip(years_tc_list,annual_rate_tc_list,stations_tc_list))
zipped_ar_total_list=list(zip(years_ar_list,annual_rate_ar_list,stations_ar_list))
zipped_other_total_list=list(zip(years_other_list,annual_rate_other_list,stations_other_list))

trends_list_t1=[]
trends_list_t2=[]
trends_list_t3=[]
trends_list_t4=[]
trends_list_t5=[]
trends_list_t6=[]
trends_list_t7=[]
trends_list_t8=[]
trends_list_t9=[]
trends_list_t10=[]
trends_list_t11=[]

trends_list_tc=[]
trends_list_ar=[]
trends_list_other=[]

sigs_list_t1=[]
sigs_list_t2=[]
sigs_list_t3=[]
sigs_list_t4=[]
sigs_list_t5=[]
sigs_list_t6=[]
sigs_list_t7=[]
sigs_list_t8=[]
sigs_list_t9=[]
sigs_list_t10=[]
sigs_list_t11=[]

sigs_list_tc=[]
sigs_list_ar=[]
sigs_list_other=[]

sigs_list_t1a=[]
sigs_list_t2a=[]
sigs_list_t3a=[]
sigs_list_t4a=[]
sigs_list_t5a=[]
sigs_list_t6a=[]
sigs_list_t7a=[]
sigs_list_t8a=[]
sigs_list_t9a=[]
sigs_list_t10a=[]
sigs_list_t11a=[]

sigs_list_tca=[]
sigs_list_ara=[]
sigs_list_othera=[]

size_list_t1=[]
size_list_t2=[]
size_list_t3=[]
size_list_t4=[]
size_list_t5=[]
size_list_t6=[]
size_list_t7=[]
size_list_t8=[]
size_list_t9=[]
size_list_t10=[]
size_list_t11=[]
size_list_tc=[]
size_list_ar=[]
size_list_other=[]

means_t1_list=[]
means_t2_list=[]
means_t3_list=[]
means_t4_list=[]
means_t5_list=[]
means_t6_list=[]
means_t7_list=[]
means_t8_list=[]
means_t9_list=[]
means_t10_list=[]
means_t11_list=[]

means_tc_list=[]
means_ar_list=[]
means_other_list=[]

yrs_neusa=np.arange(1980,2020,1)
for j in range(len(stations_station)):
    print(j)
    station=stations_station[j]
    
    select_t1=[x for x in zipped_t1_total_list if x[2]==station]
    sorted_years_t1=sorted(select_t1,key=lambda x:x[0])
    time_series_t1=[x[1] for x in sorted_years_t1]
    print(time_series_t1)

    select_t2=[x for x in zipped_t2_total_list if x[2]==station]
    sorted_years_t2=sorted(select_t2,key=lambda x:x[0])
    time_series_t2=[x[1] for x in sorted_years_t2]
    
    select_t3=[x for x in zipped_t3_total_list if x[2]==station]
    sorted_years_t3=sorted(select_t3,key=lambda x:x[0])
    time_series_t3=[x[1] for x in sorted_years_t3]
    
    select_t4=[x for x in zipped_t4_total_list if x[2]==station]
    sorted_years_t4=sorted(select_t4,key=lambda x:x[0])
    time_series_t4=[x[1] for x in sorted_years_t4]
    
    select_t5=[x for x in zipped_t5_total_list if x[2]==station]
    sorted_years_t5=sorted(select_t5,key=lambda x:x[0])
    time_series_t5=[x[1] for x in sorted_years_t5]
    
    select_t6=[x for x in zipped_t6_total_list if x[2]==station]
    sorted_years_t6=sorted(select_t6,key=lambda x:x[0])
    time_series_t6=[x[1] for x in sorted_years_t6]
    
    select_t7=[x for x in zipped_t7_total_list if x[2]==station]
    sorted_years_t7=sorted(select_t7,key=lambda x:x[0])
    time_series_t7=[x[1] for x in sorted_years_t7]
    
    select_t8=[x for x in zipped_t8_total_list if x[2]==station]
    sorted_years_t8=sorted(select_t8,key=lambda x:x[0])
    time_series_t8=[x[1] for x in sorted_years_t8]
    
    select_t9=[x for x in zipped_t9_total_list if x[2]==station]
    sorted_years_t9=sorted(select_t9,key=lambda x:x[0])
    time_series_t9=[x[1] for x in sorted_years_t9]
    
    select_t10=[x for x in zipped_t10_total_list if x[2]==station]
    sorted_years_t10=sorted(select_t10,key=lambda x:x[0])
    time_series_t10=[x[1] for x in sorted_years_t10]
    
    select_t11=[x for x in zipped_t11_total_list if x[2]==station]
    sorted_years_t11=sorted(select_t11,key=lambda x:x[0])
    time_series_t11=[x[1] for x in sorted_years_t11]
    
    select_tc=[x for x in zipped_tc_total_list if x[2]==station]
    sorted_years_tc=sorted(select_tc,key=lambda x:x[0])
    time_series_tc=[x[1] for x in sorted_years_tc]
    
    select_ar=[x for x in zipped_ar_total_list if x[2]==station]
    sorted_years_ar=sorted(select_ar,key=lambda x:x[0])
    time_series_ar=[x[1] for x in sorted_years_ar]
    
    select_other=[x for x in zipped_other_total_list if x[2]==station]
    sorted_years_other=sorted(select_other,key=lambda x:x[0])
    time_series_other=[x[1] for x in sorted_years_other]
    
    #---------
    
    annual_sum_mean_t1=float(sum(time_series_t1))/float(len(time_series_t1))
    annual_sum_mean_t2=float(sum(time_series_t2))/float(len(time_series_t2))
    annual_sum_mean_t3=float(sum(time_series_t3))/float(len(time_series_t3))
    annual_sum_mean_t4=float(sum(time_series_t4))/float(len(time_series_t4))
    annual_sum_mean_t5=float(sum(time_series_t5))/float(len(time_series_t5))
    annual_sum_mean_t6=float(sum(time_series_t6))/float(len(time_series_t6))
    annual_sum_mean_t7=float(sum(time_series_t7))/float(len(time_series_t7))
    annual_sum_mean_t8=float(sum(time_series_t8))/float(len(time_series_t8))
    annual_sum_mean_t9=float(sum(time_series_t9))/float(len(time_series_t9))
    annual_sum_mean_t10=float(sum(time_series_t10))/float(len(time_series_t10))
    annual_sum_mean_t11=float(sum(time_series_t11))/float(len(time_series_t11))
    annual_sum_mean_tc=float(sum(time_series_tc))/float(len(time_series_tc))
    annual_sum_mean_ar=float(sum(time_series_ar))/float(len(time_series_ar))
    annual_sum_mean_other=float(sum(time_series_other))/float(len(time_series_other))
    
    means_t1_list.append(annual_sum_mean_t1)
    means_t2_list.append(annual_sum_mean_t2)
    means_t3_list.append(annual_sum_mean_t3)
    means_t4_list.append(annual_sum_mean_t4)
    means_t5_list.append(annual_sum_mean_t5)
    means_t6_list.append(annual_sum_mean_t6)
    means_t7_list.append(annual_sum_mean_t7)
    means_t8_list.append(annual_sum_mean_t8)
    means_t9_list.append(annual_sum_mean_t9)
    means_t10_list.append(annual_sum_mean_t10)
    means_t11_list.append(annual_sum_mean_t11)
    
    means_tc_list.append(annual_sum_mean_tc)
    means_ar_list.append(annual_sum_mean_ar)
    means_other_list.append(annual_sum_mean_other)
    
    nonzero=len([x for x in time_series_t1 if x>0])
    if nonzero>=5:
        size_list_t1.append(1)
    else:
        size_list_t1.append(0)
    trend=mk_test(time_series_t1,alpha=0.05)[0]
    trenda=mk_test(time_series_t1,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_t1)[0]
    slope_percent=slope/annual_sum_mean_t1*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_t1.append(slope_percent)
    sigs_list_t1.append(sig)
    sigs_list_t1a.append(siga)
    
    nonzero=len([x for x in time_series_t2 if x>0])
    if nonzero>=5:
        size_list_t2.append(1)
    else:
        size_list_t2.append(0)
    trend=mk_test(time_series_t2,alpha=0.05)[0]
    trenda=mk_test(time_series_t2,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_t2)[0]
    slope_percent=slope/annual_sum_mean_t2*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_t2.append(slope_percent)
    sigs_list_t2.append(sig)
    sigs_list_t2a.append(siga)
    
    nonzero=len([x for x in time_series_t3 if x>0])
    if nonzero>=5:
        size_list_t3.append(1)
    else:
        size_list_t3.append(0)
    trend=mk_test(time_series_t3,alpha=0.05)[0]
    trenda=mk_test(time_series_t3,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_t3)[0]
    slope_percent=slope/annual_sum_mean_t3*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_t3.append(slope_percent)
    sigs_list_t3.append(sig)
    sigs_list_t3a.append(siga)
    
    nonzero=len([x for x in time_series_t4 if x>0])
    if nonzero>=5:
        size_list_t4.append(1)
    else:
        size_list_t4.append(0)
    trend=mk_test(time_series_t4,alpha=0.05)[0]
    trenda=mk_test(time_series_t4,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_t4)[0]
    slope_percent=slope/annual_sum_mean_t4*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_t4.append(slope_percent)
    sigs_list_t4.append(sig)
    sigs_list_t4a.append(siga)
    
    nonzero=len([x for x in time_series_t5 if x>0])
    if nonzero>=5:
        size_list_t5.append(1)
    else:
        size_list_t5.append(0)
    trend=mk_test(time_series_t5,alpha=0.05)[0]
    trenda=mk_test(time_series_t5,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_t5)[0]
    slope_percent=slope/annual_sum_mean_t5*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_t5.append(slope_percent)
    sigs_list_t5.append(sig)
    sigs_list_t5a.append(siga)
    
    nonzero=len([x for x in time_series_t6 if x>0])
    if nonzero>=5:
        size_list_t6.append(1)
    else:
        size_list_t6.append(0)
    trend=mk_test(time_series_t6,alpha=0.05)[0]
    trenda=mk_test(time_series_t6,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_t6)[0]
    slope_percent=slope/annual_sum_mean_t6*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_t6.append(slope_percent)
    sigs_list_t6.append(sig)
    sigs_list_t6a.append(siga)
    
    nonzero=len([x for x in time_series_t7 if x>0])
    if nonzero>=5:
        size_list_t7.append(1)
    else:
        size_list_t7.append(0)
    trend=mk_test(time_series_t7,alpha=0.05)[0]
    trenda=mk_test(time_series_t7,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_t7)[0]
    slope_percent=slope/annual_sum_mean_t7*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_t7.append(slope_percent)
    sigs_list_t7.append(sig)
    sigs_list_t7a.append(siga)
    
    nonzero=len([x for x in time_series_t8 if x>0])
    if nonzero>=5:
        size_list_t8.append(1)
    else:
        size_list_t8.append(0)
    trend=mk_test(time_series_t8,alpha=0.05)[0]
    trenda=mk_test(time_series_t8,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_t8)[0]
    slope_percent=slope/annual_sum_mean_t8*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_t8.append(slope_percent)
    sigs_list_t8.append(sig)
    sigs_list_t8a.append(siga)
    
    nonzero=len([x for x in time_series_t9 if x>0])
    if nonzero>=5:
        size_list_t9.append(1)
    else:
        size_list_t9.append(0)
    trend=mk_test(time_series_t9,alpha=0.05)[0]
    trenda=mk_test(time_series_t9,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_t9)[0]
    slope_percent=slope/annual_sum_mean_t9*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_t9.append(slope_percent)
    sigs_list_t9.append(sig)
    sigs_list_t9a.append(siga)
    
    nonzero=len([x for x in time_series_t10 if x>0])
    if nonzero>=5:
        size_list_t10.append(1)
    else:
        size_list_t10.append(0)
    trend=mk_test(time_series_t10,alpha=0.05)[0]
    trenda=mk_test(time_series_t10,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_t10)[0]
    slope_percent=slope/annual_sum_mean_t10*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_t10.append(slope_percent)
    sigs_list_t10.append(sig)
    sigs_list_t10a.append(siga)
    
    nonzero=len([x for x in time_series_t11 if x>0])
    if nonzero>=5:
        size_list_t11.append(1)
    else:
        size_list_t11.append(0)
    trend=mk_test(time_series_t11,alpha=0.05)[0]
    trenda=mk_test(time_series_t11,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_t11)[0]
    slope_percent=slope/annual_sum_mean_t11*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_t11.append(slope_percent)
    sigs_list_t11.append(sig)
    sigs_list_t11a.append(siga)
    
    nonzero=len([x for x in time_series_tc if x>0])
    if nonzero>=5:
        size_list_tc.append(1)
    else:
        size_list_tc.append(0)
    trend=mk_test(time_series_tc,alpha=0.05)[0]
    trenda=mk_test(time_series_tc,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_tc)[0]
    slope_percent=slope/annual_sum_mean_tc*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_tc.append(slope_percent)
    sigs_list_tc.append(sig)
    sigs_list_tca.append(siga)
    
    nonzero=len([x for x in time_series_ar if x>0])
    if nonzero>=5:
        size_list_ar.append(1)
    else:
        size_list_ar.append(0)
    trend=mk_test(time_series_ar,alpha=0.05)[0]
    trenda=mk_test(time_series_ar,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_ar)[0]
    slope_percent=slope/annual_sum_mean_ar*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_ar.append(slope_percent)
    sigs_list_ar.append(sig)
    sigs_list_ara.append(siga)
    
    nonzero=len([x for x in time_series_other if x>0])
    if nonzero>=5:
        size_list_other.append(1)
    else:
        size_list_other.append(0)
    trend=mk_test(time_series_other,alpha=0.05)[0]
    trenda=mk_test(time_series_other,alpha=0.1)[0]
    slope=st.linregress(yrs_neusa,time_series_other)[0]
    slope_percent=slope/annual_sum_mean_other*100.
    if trend=='increasing':
        sig=1
    elif trend=='decreasing':
        sig=-1
    else:
        sig=0  
    if trenda=='increasing':
        siga=1
    elif trenda=='decreasing':
        siga=-1
    else:
        siga=0
    trends_list_other.append(slope_percent)
    sigs_list_other.append(sig)
    sigs_list_othera.append(siga)

In [ ]:
clon=-70
clat=35
proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
fig = plt.figure(figsize=(20,16))
ax=plt.subplot(1,1,1,projection=proj_map)

ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
ax.set_extent([-84,-66,36,48],crs=ccrs.PlateCarree())

sigs_list=sigs_list_t11.copy()
sigs_lista=sigs_list_t11a.copy()
trends_list=trends_list_t11.copy()
size_list=size_list_t11.copy()

print(size_list)

if 1==1:
    for i in range(len(sigs_list)):
        if size_list[i]==0:
            trends_list[i]=np.nan
        
for i in range(len(sigs_lista)):
    sig=sigs_lista[i]
    if sig in [-1,1] and size_list[i]==1:#convert to mm
        ax.plot(lons_station[i],lats_station[i],transform=ccrs.PlateCarree(),marker='o',markerfacecolor='None',markeredgecolor='grey',markeredgewidth=2,markersize=28)
    
for i in range(len(sigs_list)):
    sig=sigs_list[i]
    if sig in [-1,1] and size_list[i]==1:#convert to mm
        ax.plot(lons_station[i],lats_station[i],transform=ccrs.PlateCarree(),marker='o',markerfacecolor='None',markeredgecolor='k',markeredgewidth=2,markersize=28)
    
# *must* call draw in order to get the axis boundary used to add ticks:
fig.canvas.draw()

#zipped_t1=list(zip(dates,ar_yesno4))#ar-related = green
#zipped_t2=list(zip(dates,tc_linked_ar_yesno4))
#zipped_t3=list(zip(dates,tc_remnant_linked_ar_yesno4))
#zipped_t4=list(zip(dates,tc_yesno4))#tc-related = blue
#zipped_t5=list(zip(dates,tc_ar_combo_yesno4))
#zipped_t6=list(zip(dates,tc_remnant_ar_combo_yesno4))
#zipped_t7=list(zip(dates,tc_remnants_yesno4))
#zipped_t8=list(zip(dates,ivt_yesno4))#other IVT-related = grey
#zipped_t9=list(zip(dates,tc_linked_ivt_yesno4))
#zipped_t10=list(zip(dates,tc_remnant_linked_ivt_yesno4))
#zipped_t11=list(zip(dates,other_yesno4))#unspecified = brown

# Define gridline locations and draw the lines using cartopy's built-in gridliner:
xticks = [-90,-85,-80,-75,-70,-65,-60,-50]
yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
ax.tick_params(labelsize=26)
# Label the end-points of the gridlines using the custom tick makers:
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)
cax=ax.scatter(lons_station,lats_station,s=240,c=trends_list,transform=ccrs.PlateCarree(),vmin=-4.5,vmax=4.5,cmap=plt.cm.seismic_r,zorder=10)
for i in range(len(trends_list)):
    lon=lons_station[i]
    lat=lats_station[i]
    size=size_list[i]
    if size==0:
    #if 1==1:
        if trends_list[i]>0:
            print('POS')
            ax.plot(lon,lat,marker='+',transform=ccrs.PlateCarree(),markersize=8,color='k')
        if trends_list[i]<0:
            print('NEG')
            ax.plot(lon,lat,marker='_',transform=ccrs.PlateCarree(),markersize=8,color='k')
cbar=plt.colorbar(cax,pad=0,extend='both',fraction=0.046)
cbar.ax.tick_params(labelsize=28)
cbar.set_label('% yr$^{-1}$',fontsize=28,rotation=90,labelpad=15)
ax.set_title('Trend',fontsize=46,pad=10)
#props = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
#ax.text(0.01, 0.99,'n='+str(len(dates_list)), transform=ax.transAxes, fontsize=28,verticalalignment='top', bbox=props)
plt.show()

In [ ]:
fig.savefig(dir+'two_week_paper_geotrends_43.png')

In [ ]:
clon=-70
clat=35
proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
fig = plt.figure(figsize=(20,16))
ax=plt.subplot(1,1,1,projection=proj_map)

ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
ax.set_extent([-84,-66,36,48],crs=ccrs.PlateCarree())

means_list=means_t11_list

# *must* call draw in order to get the axis boundary used to add ticks:
fig.canvas.draw()

# Define gridline locations and draw the lines using cartopy's built-in gridliner:
xticks = [-90,-85,-80,-75,-70,-65,-60,-50]
yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
ax.tick_params(labelsize=26)
# Label the end-points of the gridlines using the custom tick makers:
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)
cax=ax.scatter(lons_station,lats_station,s=240,c=means_list,transform=ccrs.PlateCarree(),vmin=0,vmax=50,cmap=plt.cm.Greens,zorder=10)

#cbar=plt.colorbar(cax,pad=0,extend='max',fraction=0.046,ticks=[10,20,30,40,50,60,70])
#cbar=plt.colorbar(cax,pad=0,extend='max',fraction=0.046,ticks=[4,8,12,16,20])
#cbar=plt.colorbar(cax,pad=0,extend='max',fraction=0.046,ticks=[8,16,24,32,40])
cbar=plt.colorbar(cax,pad=0,extend='max',fraction=0.046,ticks=[10,20,30,40,50])
cbar.ax.tick_params(labelsize=28)
cbar.set_label('mm yr$^{-1}$',fontsize=28,rotation=90,labelpad=15)
#ax.set_title('Annual rate',fontsize=46,pad=10)
ax.set_title('Winter',fontsize=46,pad=15)
#props = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
#ax.text(0.01, 0.99,'n='+str(len(dates_list)), transform=ax.transAxes, fontsize=28,verticalalignment='top', bbox=props)
plt.show()

In [ ]:
fig.savefig(dir+'two_week_paper_geotrends_41a.png')